<a href="https://colab.research.google.com/github/daycardoso/PredictCost/blob/main/PredicCostRandomForestRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Add `%load_ext cuml.accel` before importing sklearn to speed up operations using GPU


In [2]:
# import pandas as pd
# # Carregar o datset unificado
# df_unificado = pd.read_csv('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/df_unificado.csv')

# # matriz contendo os atributos
# X = df_unificado.iloc[:, :-1]

# # vetor contendo o custo, ou seja, a ultima coluna
# y = df_unificado.iloc[:, -1]

# # nome de cada atributo
# feature_names = df_unificado.columns[:-1]

# # nome de cada classe
# target_names = df_unificado.columns[-1]

# print(f"Dimensões de X: {X.shape}\n")
# print(f"Dimensões de y: {y.shape}\n")
# print(f"Nomes dos atributos: {feature_names}\n")
# print(f"Nomes das classes: {target_names}")
# print(f"Tamanho do dataset: {len(df_unificado)}")


# import cuml
# import cudf
# import cupy as cp
# import joblib
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from cuml.ensemble import RandomForestRegressor
# from sklearn.model_selection import cross_validate, RepeatedKFold
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import (
#     r2_score, mean_squared_error, mean_absolute_error,
#     mean_absolute_percentage_error, median_absolute_error,
#     max_error, explained_variance_score
# )
# from IPython.display import display

# # Convertendo os dados para cudf (GPU)
# X_cudf = cudf.DataFrame.from_pandas(X)
# y_cudf = cudf.Series(y)

# # Hold-out
# X_train_full, X_test, y_train_full, y_test = train_test_split(
#     X_cudf, y_cudf, test_size=0.3, random_state=42
# )

# # Treinamento
# modelo = RandomForestRegressor(
#     random_state=42,
#     max_depth=20,
#     n_estimators=100,
#     # n_streams=-1
# )
# modelo.fit(X_train_full, y_train_full)

# # Predição no hold-out (converte para numpy)
# y_pred_gpu = modelo.predict(X_test)
# # Change .to_array() to .to_numpy() to convert cudf.Series to numpy array
# y_pred = y_pred_gpu.to_numpy()
# # Change .to_array() to .to_numpy() to convert cudf.Series to numpy array
# y_test_cpu = y_test.to_numpy()

# # 5×5 CV repetida: balanceia viés x variância na estimação
# cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# scoring = {
#     'R2': 'r2',
#     'MSE': 'neg_mean_squared_error',
#     'MAE': 'neg_mean_absolute_error',
#     # 'MAPE': 'neg_mean_absolute_percentage_error',
#     'MedAE': 'neg_median_absolute_error',
#     'MaxE': 'max_error',
#     'EVS': 'explained_variance',
# }

# cv_results = cross_validate(
#     modelo, X_train_full, y_train_full,
#     cv=cv, scoring=scoring, return_train_score=True#, n_jobs=-1
# )

# cv_results_df = pd.DataFrame(cv_results)

# # Salvamento (conversão para compatibilidade com joblib)
# full_results = {
#     'pipeline': modelo,
#     'X_test':   X_test.to_pandas(),
#     'y_test':   y_test_cpu,
#     'y_pred':   y_pred,
#     'cv_results': cv_results_df,
#     'feature_names': X.columns.tolist()
# }
# joblib.dump(full_results, '/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/randon_forest_regressor_full_results.pkl')

# # Carregar resultados
# res = joblib.load('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/randon_forest_regressor_full_results.pkl')
# model         = res['pipeline']
# X_test        = res['X_test']
# y_true        = res['y_test']
# # Change .to_array() to .to_numpy() to convert cudf.Series to numpy array
# y_pred        = res.get('y_pred', model.predict(cudf.DataFrame.from_pandas(X_test)).to_numpy())
# feature_names = res.get('feature_names', getattr(X_test, 'columns', None))

# # 1) Exibir métricas
# metrics = {
#     'R² Score'          : r2_score(y_true, y_pred),
#     'MSE'               : mean_squared_error(y_true, y_pred),
#     'MAE'               : mean_absolute_error(y_true, y_pred),
#     'MAPE'              : mean_absolute_percentage_error(y_true, y_pred),
#     'Median AE'         : median_absolute_error(y_true, y_pred),
#     'Max Error'         : max_error(y_true, y_pred),
#     'Explained Variance': explained_variance_score(y_true, y_pred)
# }
# df_metrics = pd.DataFrame.from_dict(metrics, orient='index', columns=['Valor']).round(4)
# display(df_metrics)

# # 2) Gráfico real vs predito
# plt.figure()
# plt.scatter(y_true, y_pred, alpha=0.5)
# min_val, max_val = np.min([y_true.min(), y_pred.min()]), np.max([y_true.max(), y_pred.max()])
# plt.plot([min_val, max_val], [min_val, max_val], 'r--')
# plt.xlabel('Valor Real')
# plt.ylabel('Valor Predito')
# plt.title('Real vs Predito')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# # 3) Histograma de resíduos
# residuals = y_true - y_pred
# plt.figure()
# plt.hist(residuals, bins=50)
# plt.xlabel('Resíduo (Real - Predito)')
# plt.title('Histograma de Resíduos')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# # 4) Boxplot de resíduos por quartil
# df_r = pd.DataFrame({'Real': y_true, 'Resíduo': residuals})
# df_r['Quartil'] = pd.qcut(df_r['Real'], 4, labels=[1, 2, 3, 4])
# groups = [df_r[df_r['Quartil'] == q]['Resíduo'] for q in sorted(df_r['Quartil'].unique())]
# plt.figure()
# plt.boxplot(groups, labels=sorted(df_r['Quartil'].unique()))
# plt.xlabel('Quartil de Valor Real')
# plt.ylabel('Resíduo')
# plt.title('Boxplot de Resíduos por Quartil')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# # 5) Importância das features (compatível com cuML)
# if hasattr(model, 'feature_importances_') and feature_names is not None:
#     # Change .to_array() to .to_numpy() to convert cudf.Series to numpy array
#     fi = pd.Series(model.feature_importances_.to_numpy(), index=feature_names).sort_values(ascending=False)
#     plt.figure(figsize=(10, 4))
#     fi.plot(kind='bar')
#     plt.ylabel('Importância')
#     plt.title('Importância das Features')
#     plt.tight_layout()
#     plt.show()

In [ ]:
import cuml
import pandas as pd
import numpy as np
import cudf
from cuml.ensemble import RandomForestRegressor
# Correct the import for RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    mean_absolute_percentage_error, median_absolute_error,
    max_error, explained_variance_score
)
import matplotlib.pyplot as plt
import joblib
import json

# 1. Carregamento do dataset
df_unificado = pd.read_csv('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/df_unificado.csv')
X = df_unificado.iloc[:, :-1]
y = df_unificado.iloc[:, -1]
feature_names = df_unificado.columns[:-1]
target_name = df_unificado.columns[-1]
print(f"Dimensões de X: {X.shape}")
print(f"Dimensões de y: {y.shape}")
print(f"Nomes dos atributos: {feature_names.tolist()}")
print(f"Nome da variável target: {target_name}")
print(f"Tamanho do dataset: {len(df_unificado)}")

# 2. Conversão para cudf
X_cudf = cudf.DataFrame.from_pandas(X)
y_cudf = cudf.Series(y)

# 3. Split Hold-out para avaliação final (antes do tuning)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_cudf, y_cudf, test_size=0.3, random_state=42
)

# 4. Amostragem para tuning (~100k exemplos)
amostra_pct = min(100_000 / len(X_train_full), 1.0)
X_tune, _, y_tune, _ = train_test_split(
    X_train_full, y_train_full, train_size=amostra_pct, random_state=42
)
print(f"Amostra para tuning: {len(X_tune)} exemplos.")

# 5. Espaço de busca RandomizedSearchCV
from scipy.stats import randint
param_dist = {
    'n_estimators': randint(50, 250),
    'max_depth': randint(3, 25),
    'max_features': ['sqrt', 'log2', 0.2, 0.5],
    'min_samples_leaf': randint(1, 5),
    'bootstrap': [True, False]
}

rf = RandomForestRegressor(random_state=42)
search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=3,
    random_state=42,
    verbose=2
)

# 6. Tuning dos hiperparâmetros
print("\nIniciando tuning...")
search.fit(X_tune, y_tune)
print("Melhores parâmetros encontrados:", search.best_params_)

# Early stopping "manual":
scores = -search.cv_results_['mean_test_score']
improvements = np.diff(scores)
if len(improvements) > 0 and np.abs(improvements).max() < 0.01 * np.mean(scores):
    print("Melhorias marginais. Early stopping aplicado.")

# 7. Treinamento do modelo final com todos os dados de treino
print("\nTreinando modelo final com todos os dados disponíveis...")
best_params = search.best_params_
rf_final = RandomForestRegressor(**best_params, random_state=42)
rf_final.fit(X_train_full, y_train_full)

# 8. Avaliação no hold-out
y_pred_gpu = rf_final.predict(X_test)
y_pred = y_pred_gpu.to_numpy()
y_test_cpu = y_test.to_numpy()

metrics = {
    'R² Score'          : r2_score(y_test_cpu, y_pred),
    'MSE'               : mean_squared_error(y_test_cpu, y_pred),
    'MAE'               : mean_absolute_error(y_test_cpu, y_pred),
    'MAPE'              : mean_absolute_percentage_error(y_test_cpu, y_pred),
    'Median AE'         : median_absolute_error(y_test_cpu, y_pred),
    'Max Error'         : max_error(y_test_cpu, y_pred),
    'Explained Variance': explained_variance_score(y_test_cpu, y_pred)
}
df_metrics = pd.DataFrame.from_dict(metrics, orient='index', columns=['Valor']).round(4)

# 9. Relatório visual
plt.figure()
plt.scatter(y_test_cpu, y_pred, alpha=0.5)
min_val, max_val = np.min([y_test_cpu.min(), y_pred.min()]), np.max([y_test_cpu.max(), y_pred.max()])
plt.plot([min_val, max_val], [min_val, max_val], 'r--')
plt.xlabel('Valor Real')
plt.ylabel('Valor Predito')
plt.title('Real vs Predito')
plt.grid(True)
plt.tight_layout()
plt.show()

residuals = y_test_cpu - y_pred
plt.figure()
plt.hist(residuals, bins=50)
plt.xlabel('Resíduo (Real - Predito)')
plt.title('Histograma de Resíduos')
plt.grid(True)
plt.tight_layout()
plt.show()

df_r = pd.DataFrame({'Real': y_test_cpu, 'Resíduo': residuals})
df_r['Quartil'] = pd.qcut(df_r['Real'], 4, labels=[1, 2, 3, 4])
groups = [df_r[df_r['Quartil'] == q]['Resíduo'] for q in sorted(df_r['Quartil'].unique())]
plt.figure()
plt.boxplot(groups, labels=sorted(df_r['Quartil'].unique()))
plt.xlabel('Quartil de Valor Real')
plt.ylabel('Resíduo')
plt.title('Boxplot de Resíduos por Quartil')
plt.grid(True)
plt.tight_layout()
plt.show()

if hasattr(rf_final, 'feature_importances_') and feature_names is not None:
    fi = pd.Series(rf_final.feature_importances_.to_numpy(), index=feature_names).sort_values(ascending=False)
    plt.figure(figsize=(10, 4))
    fi.plot(kind='bar')
    plt.ylabel('Importância')
    plt.title('Importância das Features')
    plt.tight_layout()
    plt.show()

# 10. Salvar resultados e parâmetros
full_results = {
    'pipeline': rf_final,
    'X_test':   X_test.to_pandas(),
    'y_test':   y_test_cpu,
    'y_pred':   y_pred,
    'metrics':  df_metrics,
    'best_params': best_params,
    'feature_names': X.columns.tolist()
}
joblib.dump(full_results, '/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/rf_regressor_final_full_results.pkl')

print("\nRelatório Final:")
print("Melhores hiperparâmetros:", best_params)
print("\nMétricas no Hold-out:")
print(df_metrics)

# Top 10 features
feature_importances = None
if hasattr(rf_final, 'feature_importances_') and feature_names is not None:
    fi = pd.Series(rf_final.feature_importances_.to_numpy(), index=feature_names).sort_values(ascending=False)
    top_features = fi.head(10).to_dict()
    feature_importances = fi.to_dict()
else:
    top_features = None

# Relatório JSON
relatorio = {
    'dataset': {
        'total_samples': int(len(df_unificado)),
        'n_features': int(X.shape[1]),
        'feature_names': feature_names.tolist(),
        'target_name': target_name
    },
    'tuning': {
        'sample_size': int(len(X_tune)),
        'param_dist': {k: (str(v) if not isinstance(v, (int, float, list, tuple, dict)) else v) for k, v in param_dist.items()},
        'n_iter': int(search.n_iter),
        'cv': int(search.cv),
        'random_state': int(search.random_state),
    },
    'best_params': best_params,
    'metrics': df_metrics['Valor'].to_dict(),
    'feature_importances_top10': top_features,
    'full_feature_importances': feature_importances if feature_importances is not None else None,
    'predictions': {
        'real': y_test_cpu.tolist()[:1000],     # Salve apenas os 1000 primeiros para não ficar gigante
        'predicted': y_pred.tolist()[:1000],
        'residuals': residuals.tolist()[:1000]
    },
    'artifacts': {
        'model_pkl': '/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/rf_regressor_final_full_results.pkl'
    }
}

# Salvando relatório JSON
relatorio_path = '/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/relatorio_rf_final.json'
with open(relatorio_path, 'w') as f:
    json.dump(relatorio, f, indent=4)

print(f"Relatório salvo em: {relatorio_path}")


Dimensões de X: (6336328, 8)
Dimensões de y: (6336328,)
Nomes dos atributos: ['x0', 'y0', 'theta0', 'beta0', 'xf', 'yf', 'thetaf', 'betaf']
Nome da variável target: cost
Tamanho do dataset: 6336328


/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)
/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)
/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:368: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return init_func(self, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)


Amostra para tuning: 100000 exemplos.

Iniciando tuning...
Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/us

[CV] END bootstrap=True, max_depth=29, max_features=sqrt, min_samples_leaf=3, n_estimators=156; total time=   1.6s


...

[CV] END bootstrap=True, max_depth=25, max_features=sqrt, min_samples_leaf=2, n_estimators=160; total time=   0.7s


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/us

[CV] END bootstrap=True, max_depth=25, max_features=sqrt, min_samples_leaf=2, n_estimators=160; total time=   0.7s


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/us

[CV] END bootstrap=True, max_depth=25, max_features=sqrt, min_samples_leaf=2, n_estimators=160; total time=   0.7s
Melhores parâmetros encontrados: {'bootstrap': True, 'max_depth': 29, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'n_estimators': 156}

Treinando modelo final com todos os dados disponíveis...
